# Environment

In [ ]:
! pip install scipy -q
! pip install lightfm -q

In [ ]:
import random
import pickle
import pandas as pd
import numpy as np
import scipy
from scipy.sparse import csr_matrix
from lightfm import LightFM
from lightfm.data import Dataset
from sklearn.metrics.pairwise import cosine_similarity

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Загрузка данных

In [ ]:
# Десериализация
with open ('/content/model_base', 'rb') as fp:
  model_base = pickle.load(fp)
with open ('/content/model_items', 'rb') as fp:
  model_items = pickle.load(fp)
with open ('/content/model_users', 'rb') as fp:
  model_users = pickle.load(fp)

with open ('/content/df_all', 'rb') as fp:
  df_all = pickle.load(fp)
with open ('/content/user_attributes', 'rb') as fp:
  user_attributes = pickle.load(fp)
with open ('/content/train_interactions', 'rb') as fp:
  train_interactions = pickle.load(fp)
with open ('/content/inv_item_mappings', 'rb') as fp:
  inv_item_mappings = pickle.load(fp)
with open ('/content/inv_user_mappings', 'rb') as fp:
  inv_user_mappings = pickle.load(fp)
with open ('/content/item_metadata', 'rb') as fp:
  item_metadata = pickle.load(fp)
with open ('/content/item_metadata_list', 'rb') as fp:
  item_metadata_list = pickle.load(fp)
with open ('/content/user_metadata', 'rb') as fp:
  user_metadata = pickle.load(fp)
with open ('/content/user_metadata_mappings_users', 'rb') as fp:
  user_metadata_mappings_users = pickle.load(fp)
with open ('/content/user_mappings', 'rb') as fp:
  user_mappings = pickle.load(fp)
with open ('/content/item_mappings', 'rb') as fp:
  item_mappings = pickle.load(fp)
with open ('/content/train', 'rb') as fp:
  train = pickle.load(fp)
with open ('/content/item_metadata_mappings_items', 'rb') as fp:
  item_metadata_mappings_items = pickle.load(fp)

In [ ]:
n=0
while n<4:
  if n%2 ==0:
    user = random.choice(list(set(inv_user_mappings.keys())))
    # Создадим матрицу всех пользователей и элементов, чтобы получить для них прогнозы
    n_users, n_items = train_interactions.shape

      # Используем lightFM to create predictions for all users and all items
    scoring_user_ids = np.concatenate([np.full((n_items, ), i) for i in range(n_users)]) # повторим user ID для всех проб
    scoring_item_ids = np.concatenate([np.arange(n_items) for i in range(n_users)]) # повторим весь диапазон идентификаторов item IDs x количество user
    scores = model_base.predict(user_ids = scoring_user_ids,
                                          item_ids = scoring_item_ids)
    scores = scores.reshape(-1, n_items) # получим одну строку на каждого user
      # Получим информацию о предыдущих покупках для каждого user
    previous = np.array(train_interactions.todense())
    print(f'User: {user}')
    print(f'По данному user имеется история взаимодействий с items.\nРекомендации строятся на основе коллаборативной фильтрации')
    # 3 лучших прогнозов для каждого пользователя
    k=3
    top_3 = np.argsort(-scores, axis=1) [::, :k]
    print("Предыдущие покупки данного user:", *[inv_item_mappings.get(key) for key in np.array(range(previous.shape[1]))[previous[user]>0]], sep="\n")
    print("Top 3 recommendations:", *sorted(zip([inv_item_mappings.get(key) for key in top_3[user]], range(k)), key = lambda x: x[1]), sep="\n")
    # Удалим ранее купленные items из прогнозов
    top_3_new  = np.argsort(-(scores-(previous*999999)), axis=1)[::, :k] # вычтем предыдущие покупки из прогнозов
    print("Top 3 new items recommendations:", *sorted(zip([inv_item_mappings.get(key) for key in top_3_new[user]], range(k)), key = lambda x: x[1]), sep="\n")
    print('\n')
  else:
    user = random.randrange(10000000)
    try:
      п = previous[user]
      print(f'User: {user}')
      print(f'По данному user имеется история взаимодействий с items.\nРекомендации строятся на основе коллаборативной фильтрации')
      k=3
      top_3 = np.argsort(-scores, axis=1) [::, :k]
      print("Предыдущие покупки данного user:", *[inv_item_mappings.get(key) for key in np.array(range(previous.shape[1]))[previous[user]>0]], sep="\n")
      print("Top 3 recommendations:", *sorted(zip([inv_item_mappings.get(key) for key in top_3[user]], range(k)), key = lambda x: x[1]), sep="\n")
        # Удалим ранее купленные items из прогнозов
      top_3_new  = np.argsort(-(scores-(previous*999999)), axis=1)[::, :k] # вычтем предыдущие покупки из прогнозов
      print("Top 3 new items recommendations:", *sorted(zip([inv_item_mappings.get(key) for key in top_3_new[user]], range(k)), key = lambda x: x[1]), sep="\n")
      print('\n')
    except:
      print(f'User: {user}')
      print(f'По данному user нет данных о взаимодействии с items.\nРекомендации строятся на основе матричного разложения')
      print(f'Для построения рекомендаций сгенерируем случайные признаки user:')
      new_user_attriutes = random.sample(list(user_metadata),k=5)
      print(new_user_attriutes)
      user_indexes = [user_metadata_mappings_users.get(key) for key in new_user_attriutes]
      # Can either just weight each attribute equally
      weights = 1/len(user_indexes) # weight each metadata equally
      std_weights = [[weights] * len(new_user_attriutes)]

      # Combine the indexes we want populating with their weights
      new_user = np.zeros(len(user_metadata_mappings_users)) # create an empty array that will server as our dummy cold-user row
      np.put(new_user, user_indexes, std_weights) # update the relevant metadata attributes with the desired weights

      #  Now we can predict on this cold-user just like any other
      cold_user_preds = model_users.predict(user_ids = 0,
                                      item_ids = [*item_mappings.values()],
                                      item_features = item_metadata_list,
                                      user_features = scipy.sparse.csr_matrix(new_user))

      cold_ranks = np.argsort(-cold_user_preds)[:3]
      cold_ranks = pd.DataFrame(zip([*inv_item_mappings.values()], cold_ranks), columns = ["product_name", "rank"])
      print('Рекомендации, сформированные на основе похожести признаков users')
      print(cold_ranks.sort_values(["rank"])[:3])
      print('')
  n+=1

User: 4332
По данному user имеется история взаимодействий с items.
Рекомендации строятся на основе коллаборативной фильтрации
Предыдущие покупки данного user:
321054
Top 3 recommendations:
(461686, 0)
(48030, 1)
(7943, 2)
Top 3 new items recommendations:
(461686, 0)
(48030, 1)
(7943, 2)


User: 860445
По данному user нет данных о взаимодействии с items.
Рекомендации строятся на основе матричного разложения
Для построения рекомендаций сгенерируем случайные признаки user:
['day_of_week_3', 'day_of_week_4', 'day_of_week_5', 'Month_6', 'day_of_week_2']
Рекомендации, сформированные на основе похожести признаков users
   product_name  rank
1        178480   617
2         94371   660
0         19278  1295

User: 7574
По данному user имеется история взаимодействий с items.
Рекомендации строятся на основе коллаборативной фильтрации
Предыдущие покупки данного user:
29873
Top 3 recommendations:
(420960, 0)
(7943, 1)
(309778, 2)
Top 3 new items recommendations:
(420960, 0)
(7943, 1)
(309778, 2)




## Items feature

In [ ]:
n=0
while n<4:
  if n%2 != 0:
    itemid = random.choice(list(set(inv_item_mappings.keys())))
    # Создадим матрицу всех пользователей и элементов, чтобы получить для них прогнозы
    n_users, n_items = train_interactions.shape
    scores = model_base.predict(np.arange(n_users), np.repeat(itemid, n_users))
    # Получаем индексы пользователей, отсортированных по их вероятності взаимодействия с продуктом
    top_users_indices = np.argsort(-scores)
    print(f'Item: {itemid}')
    print(f'По данному item имеется история взаимодействий с users.\nРекомендации строятся на основе коллаборативной фильтрации')
    # Получаем индексы пользователей, отсортированных по их вероятности взаимодействия с продуктом
    top_users_indices = np.argsort(-scores)
    top_users = [inv_user_mappings[idx] for idx in top_users_indices][:3]
    print("Наиболее подходящие пользователи для продукта {}: ".format(itemid))
    for user in top_users:
        print(user)
    print('\n')
  else:
    itemid = random.randrange(1000000)
    if itemid in set(inv_item_mappings.keys()):
      # Создадим матрицу всех пользователей и элементов, чтобы получить для них прогнозы
      n_users, n_items = train_interactions.shape
      scores = model_base.predict(np.arange(n_users), np.repeat(itemid, n_users))
      # Получаем индексы пользователей, отсортированных по их вероятності взаимодействия с продуктом
      top_users_indices = np.argsort(-scores)
      print(f'Item: {itemid}')
      print(f'По данному item имеется история взаимодействий с users.\nРекомендации строятся на основе коллаборативной фильтрации')
      # Получаем индексы пользователей, отсортированных по их вероятности взаимодействия с продуктом
      top_users_indices = np.argsort(-scores)
      top_users = [inv_user_mappings[idx] for idx in top_users_indices][:3]
      print("Наиболее подходящие пользователи для продукта {}: ".format(itemid))
      for user in top_users:
          print(user)
      print('\n')
    else:
      print(f'Item: {itemid}')
      print(f'По данному item нет данных о взаимодействии с userss.\nРекомендации строятся на основе матричного разложения')
      print(f'Для построения рекомендаций сгенерируем случайные признаки item:')
      # Получим indexes for the feature combinations we want to return embeddings for
      new_item_attriutes = random.sample(list(item_metadata),k=5)
      print(new_item_attriutes)
      new_item_indexes = [item_metadata_mappings_items.get(key) for key in new_item_attriutes]
      # new_item_indexes
      print('Вначале найдем похожие items')
      # Can just weight each attribute equally
      weights = 1/len(new_item_indexes) # weight each metadata equally
      std_weights = [[weights] * len(new_item_attriutes)]

      new_item = np.zeros(len(item_metadata_mappings_items)) # create an empty array that will serve as our dummy cold-user row
      np.put(new_item, new_item_indexes, std_weights) # update the relevant metadata attributes with the desired weights

      # Convert it into a sparse matrix
      cold_item_matrix = scipy.sparse.csr_matrix(new_item)

      # Use LightFM to convert the matrix into embeddings
      cold_item_bias, cold_item_embedding = model_items.get_item_representations(cold_item_matrix)
      item_biases, item_embeddings  = model_items.get_item_representations(features = item_metadata_list)

      # Находим похожие items
      item_item_cold = pd.DataFrame(cosine_similarity(cold_item_embedding, item_embeddings).T, columns=(["cosine"]))
      item_item_cold["item_name"]=item_item_cold.index.to_series().map(inv_item_mappings)
      print(item_item_cold.sort_values(by="cosine", ascending=False)[:10])
      print("Затем определим users  подходящих по предпочтениям для данного item")
      # Create all user and item matrix to get predictions for it
      n_users, n_items = train_interactions.shape

      # Force lightFM to create predictions for all users and all items
      scoring_user_ids = np.concatenate([np.full((n_items, ), i) for i in range(n_users)]) # repeat user ID for number of prods
      scoring_item_ids = np.concatenate([np.arange(n_items) for i in range(n_users)]) # repeat entire range of item IDs x number of user
      scores = model_items.predict(user_ids = scoring_user_ids,
                                          item_ids = scoring_item_ids)
      scores = scores.reshape(-1, n_items) # get 1 row per user
      recommendations = pd.DataFrame(scores)

      # Extract the user and item representations
      user_biases, user_embeddings  = model_items.get_user_representations()
      # Create prediction score for our 'new' item
      recommendations["cold_ranking"] = ((user_embeddings @ cold_item_embedding.T + cold_item_bias).T + user_biases).T
      # recommendations.rank(axis=1, ascending=False)  # Highest value gets ranked as 1 i.e. best rec
      cold_rankings = recommendations.rank(axis=1, ascending=False)[["cold_ranking"]]
      # cold_rankings

      # Add on users
      cold_rankings["user_id"]=cold_rankings.index.to_series().map(inv_user_mappings)
      print(cold_rankings.sort_values(by="cold_ranking")[:3])
      print('')
  n+=1

Item: 340938
По данному item нет данных о взаимодействии с userss.
Рекомендации строятся на основе матричного разложения
Для построения рекомендаций сгенерируем случайные признаки item:
['159_519769', '283_30603', 'available_1', '888_1284577', '283_1128577']
Вначале найдем похожие items
        cosine  item_name
5041  0.980899      89143
3437  0.964362     122324
2240  0.956632      47000
6198  0.953952     371228
3020  0.947665     129972
2250  0.943244     379796
5966  0.939649     358984
7808  0.939256     162863
2280  0.938702     244285
2981  0.935320     128745
Затем определим users  подходящих по предпочтениям для данного item
      cold_ranking  user_id
2139           1.0   311402
9089           1.0  1307466
6386           1.0   929721

Item: 4041
По данному item имеется история взаимодействий с users.
Рекомендации строятся на основе коллаборативной фильтрации
Наиболее подходящие пользователи для продукта 4041: 
1309591
173056
903847


Item: 550671
По данному item нет данных о 